# Assignment 2: Working with Word Embeddings

In this assigment will learn how to work with word embeddings, and that with simple techniques we can implement cool things.

The assignment is divided in many parts that in most of the cases do not need code implementation. Hopefully, you would complete the assignment in short time and have great fun too. 

- __Part 0__: It is for setting up every thing, like load embeddings, normalize them etc.
- __Part 1__: We will use word embeddings for finding similar and relatead words.
- __Part 2__: We will learn scoring semantically words.
- __Part 3__: We will learn doing analogies, like `man-king` and `woman-queen`
- __Part 4__: We will learn visualizing embeddings.

## Part 0: Set up

We firtst define some helper functions for printing results and reading embeddings.

In [ ]:
def print_header(title):
    print('┌───────────────────────────────────────────────────────────────┐')
    print('│{0:^63}│'.format(title))
    print('├──────────┬─────────────────────────────┬──────────┬───────────┤')

def print_footer():
    print('└──────────┴─────────────────────────────┴──────────┴───────────┘')

def print_oov(oov):
    if len(oov) > 0:
        print('OOV: {0}'.format(', '.join(oov)))

def print_row(index, last, trg_words, knn, sim):
    if last >= index or index < 0:
        return last
    if last < index - 1:
        print('│  {0:>5}  │  {0:^25}  │  {0:>5}   │  {0:^7} │'.format('⋮'))
    word = trg_words[knn[index]]
    word = ('{0:^25}').format(word)
    print('│  {0:>6}  │  {1}  │  {2:>6}  │  {3:7.4f}  │'.format(index + 1, word, knn[index], sim[knn[index]]))
    return index

In [ ]:
import numpy as np

def read(file, threshold=0, dim=50, vocabulary=None):
    count = 400000 if threshold <= 0 else min(threshold, 400000)
    words = []
    matrix = np.empty((count, dim)) if vocabulary is None else []
    for i in range(count):
        word, vec = file.readline().split(' ', 1)
        if vocabulary is None:
            words.append(word)
            matrix[i] = np.fromstring(vec, sep=' ')
        elif word in vocabulary:
            words.append(word)
            matrix.append(np.fromstring(vec, sep=' '))
    return (words, matrix) if vocabulary is None else (words, np.array(matrix))

In [ ]:
def length_normalize(matrix):
    norms = np.sqrt(np.sum(matrix**2, axis=1))
    norms[norms == 0] = 1
    return matrix / norms[:, np.newaxis]

__Load data__

First let's load a set of 50D word vectors from GloVe. You can download them at the following [url](http://nlp.stanford.edu/data/glove.6B.zip) (1GB). The zip file includes embeddings of different dimensionality (50d, 100d, 200d, 300d) for a vocabulary of 400000 words. Decompress them and place somewhere, for example in `./embeddings/` folder

`glove_home` below specifies the location of the unzipped file. 

Variables like `matrix` and `word2ind` are used below in the notebook by different functions, so you need first load data in order to make everything work.

In [ ]:
# Read input embeddings
glove_home = 'embeddings/glove.6B.50d.txt'
embsfile = open(glove_home, encoding='utf-8', errors='surrogateescape')
words, matrix = read(embsfile)

# Length normalize embeddings so their dot product effectively computes the cosine similarity
matrix = length_normalize(matrix)

# Build word to index map
word2ind = {word: i for i, word in enumerate(words)}

## Part 1: Semantically similar/related words

In [ ]:
def knn(word, k=10):
    try:
        i = word2ind[word]
        sim = matrix[i].dot(matrix.T)
        knn = np.argsort(-sim)
    except KeyError:
        print_header('{0} (OOV)'.format(word))
        print_footer()
        print()
        return
    print_header('{0} ({1})'.format(word, i + 1))
    last = -1
    for j in range(len(knn)):
        word = words[knn[j]]
        if j < k:
            last = print_row(j, last=last, trg_words=words, knn=knn, sim=sim)
    last = print_row(len(knn)-1, last=last, trg_words=words, knn=knn, sim=sim)
    print_footer()
    print()


`knn` function retrieve the _k_ most similar/related words of the target word according to the given embedding space. The output looks like the following:

   - fist column for nearest neighbour index
   - second column for nearest neighbour word
   - third column for index of word in frequency ranking (1 is most frequent)
   - last column for cosine (1 for perfect similarity)



In [ ]:
# Show the 30 nearest neighbors
knn('pound', k=30)

### TODO1: 
Check the results for the words below. List which words you think are working well, and which ones it is failing. You can use the example in the slide (page 27) as reference.

- france, jesus, xbox, reddish, scratched, megabits 

Try any other word you fancy, and write any comment you might have. Keep a copy of the output including 30 nearest neighbours.

In [ ]:
knn('jesus',k=30)

## Part 2: Semantic orientation

The __semantic orientation__ method of [Turney and Littman 2003](http://doi.acm.org/10.1145/944012.944013) is a method for automatically scoring words along some single semantic dimension like sentiment. It works from a pair of small seed sets of words that represent two opposing points on that dimension.

In [ ]:
seed_pos = ['good', 'great', 'awesome', 'like', 'love']
seed_neg = ['bad', 'awful', 'terrible', 'hate', 'dislike']

def determine_coefficient(candidate_word, seed_pos, seed_neg):
    pos_ind = np.array([word2ind[word] for word in seed_pos])
    pos_mat = matrix[pos_ind]

    neg_ind = np.array([word2ind[word] for word in seed_neg])
    neg_mat = matrix[neg_ind]

    i = word2ind[candidate_word]

    pos_sim = np.sum(matrix[i].dot(pos_mat.T))
    neg_sim = np.sum(matrix[i].dot(neg_mat.T))

    return pos_sim - neg_sim

print(determine_coefficient('abhorrent', seed_pos, seed_neg))
print(determine_coefficient('vacations', seed_pos, seed_neg))
print(determine_coefficient('hunger', seed_pos, seed_neg))

And sort our vocabulary by its score along the axis. For now, we're only scoring frequent words, since this process can be slow.

In [ ]:
from operator import itemgetter

scored_words = [(word, determine_coefficient(word, seed_pos, seed_neg)) for word in words[1:5000]]
sorted_words = sorted(scored_words, key=itemgetter(1), reverse=True)

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

pp.pprint(sorted_words[:10])
pp.pprint(sorted_words[-10:])

### TODO 2
Spend a few minutes exploring possible seed sets for semantic dimensions other than sentiment (e.g. "animals" vs "tools"). 

- Define your semantic orientation with the two sets of seeds.
- Report (write it here in the notebook) what works, what doesn't, and why?

## Part 3: Analogy

Next, let's try to build a similar function for determining which words are likely to be good completions for an analogy. Our inputs will be a pair and a singleton word that together represent an analogy problem.

- Analogy pair:  good $\rightarrow$ best,  man $\rightarrow$ king
- Analogy problem: bad $\rightarrow$ ??,  woman $\rightarrow$ ??


Remenber from slides:

- Task: _a is to b as c is to?_
    + $a-b \approx c-d$
    + $c-a+b \approx d$
    + $argmax_{d\in V} (cos(d , c−a+b))$

In [ ]:
def analogy(pront_pair, pront_seed, k=10):
    # The function make use of embedding matrix and word indices.
    # Recall to load data and inialize matrix and word2ind variables.
    try:
        i = word2ind[pront_pair[0]]
        w1v = matrix[i]
    except KeyError:
        print_header('{0} (OOV)'.format(pront_pair[0]))
        print_footer()
        print()
        return
    try:
        i = word2ind[pront_pair[1]]
        w2v = matrix[i]
    except KeyError:
        print_header('{0} (OOV)'.format(pront_pair[1]))
        print_footer()
        print()
        return
    try:
        i = word2ind[pront_seed]
        w3v = matrix[i]
    except KeyError:
        print_header('{0} (OOV)'.format(pront_seed))
        print_footer()
        print()
        return
    
    ########### YOUR SOLUTION HERE
    knn = None # obtain k nearest words according to the analogy
    ###########

    print_header('{0} - {1} + {2}'.format(pront_pair[0], pront_pair[1], pront_seed))
    last = -1
    for j in range(len(knn)):
        word = words[knn[j]]
        if j < k:
            last = print_row(j, last=last, trg_words=words, knn=knn, sim=sim)
    last = print_row(len(knn)-1, last=last, trg_words=words, knn=knn, sim=sim)
    print_footer()
    print()


In [ ]:
prompt_pair = ('good', 'best')
prompt_seed = 'bad'
analogy(prompt_pair, prompt_seed)

prompt_pair = ('man', 'king')
prompt_seed = 'woman'
analogy(prompt_pair, prompt_seed)



### TODO3
-  The embeddings space can be used to do analogies. Please examine the formula in the of slides, and apply it to the three embeddings in the function below `analogy`. If you programmed it correctly, the following analogy should work:
    + `man:king; woman:?` (Note that, in the result list, words in the query need to be ignored)
    

- Check 10 of the analogie below and report in which position is the correct answer (discounting the words in the query, of course).

![](./analogy_exercise.png)


In [ ]:
analogy(['france', 'paris'], 'italy')

## Part 4: Visualization
Below we'll use T-SNE to visualize how our high-dimensional word vectors cluster together. T-SNE is used to project these vectors into two dimensions while preserving local stucture. Check out [this post from Christopher Olah](http://colah.github.io/posts/2014-10-Visualizing-MNIST/) to learn more about T-SNE and other ways to visualize high-dimensional data.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

# reduce size of the matrix to speed up the operations
viz_words = 500
start_ind = 1000
end_ind = start_ind+viz_words
small_ind = np.array([word2ind[word] for word in words[start_ind:end_ind]])
small_word2ind = {word : i for i, word in enumerate(words[start_ind:end_ind])}
small_matrix =  matrix[small_ind]

# Project word embeddings to two-dimensions
tsne = TSNE()
embed_tsne = tsne.fit_transform(small_matrix)

In [ ]:
fig, ax = plt.subplots(figsize=(14, 14))
plt.scatter(x=embed_tsne[:,0], y=embed_tsne[:,1], c='steelblue')
for i, word in enumerate(words[start_ind:end_ind]):
    plt.annotate(word, (embed_tsne[small_word2ind[word], 0], embed_tsne[small_word2ind[word], 1]), alpha=0.7)

### TODO4
- Sort word according to some semantic dimension (you could use the one on sentiment)
- Plot them some of them using T-SNE 
- It would help visualization coloring word-points according to their positive/negativ orientation 

# Atribution: 
Adapted by Oier Lopez de Lacalle, based on the code by Mikel Artetxe at UPV/EHU